In [3]:
## get data
import os
import wrangle

## analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

**Formula**  

- for each states:
    - **Note:** proportion of ppl get sick who are in labor force (current unemployment rate, by age)

$$unquarantine\_ change\_ rate=\sum_{i=0}^{n} \sum_{j=0}^{m}{proportion\_of\_state\_sick_{i,j}}\times{proportion\_of\_inds\_in\_state_{i,j}}$$
$$n: number \space of \space states$$
$$m: number \space of \space industries$$


- Depending on how much data we have, we could put _shock_ in the formula (financial crsis, hospital bed overflow) 
    
- quarantine_change_rate (come back to this)
    - we should not assume quarantine rate for all the states, might need diff quarantine states
    - Jenn: current quarantine measures for each states
    - WFH possibility will diff from industry
    - layoffs

- Health care cost 

## Data
- employment 2019
- covid 19 confirmed cases
- states populations :https://www.census.gov/quickfacts/NY


### employment

In [4]:
employment_ny = wrangle.get_ny_employment()

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 for Excel support Use pip or conda to install xlrd.

In [ ]:
employment_ny.head()

In [ ]:
employment_nj = wrangle.get_nj_employment()

In [ ]:
employment_nj.head()

In [ ]:
employment_ny.industry

### covid 19

In [ ]:
covid19_us = wrangle.get_covid_data(date = "03-30-2020")

In [ ]:
ny_confirmed = covid19_us.loc[covid19_us["Province_State"] == "New York", "Confirmed"].sum()

### population

In [ ]:
population_2019 = wrangle.get_population_data()

In [ ]:
population_2019.head()

In [ ]:
ny_pop = population_2019.loc[population_2019["NAME"] == "New York","POPESTIMATE2019"].values[0]

In [ ]:
ny_confirmed/ny_pop

### GDP

In [ ]:
gdp_dir = os.path.join(os.getcwd(),"data/qgdpstate0120_2.pdf")

In [ ]:
gdp_states = wrangle.get_gdp_by_states(data_dir=gdp_dir)

In [ ]:
gdp_states_inds = wrangle.get_gdp_pct_by_states_by_inds(data_dir=gdp_dir)

In [ ]:
gdp_states.head()

In [ ]:
gdp_states_inds.head()

## unquarantine_change_rate: 

In [ ]:
(employment_ny['2019']/sum(employment_ny['2019']))*(ny_confirmed/ny_pop)

In [ ]:
def get_unquarantine_change_rate():
    
    return 
    

- output

(total dollar producted per hours of work) * (total hours put in)
- pandemic in various way 
    - affect # of hours or value come out of the hours 
        - in quarn affected inds, dollar value per hours go down (nobody buys the product), entertament, macy's
        
        - num of hours: ppl get sick not able to work (especially die), reduce # of hours, tradeoff: forcing business to close, reducing dollar value. E.g. Blackstone, 
    - 

**Next:**
- use the code SEIR, along with initial condition nytimes, provide state level number of discese ineffected, by state, by industry, and then hours reduction and dollar value

- then link to state level eco data

## SEIR Model 
- by Christian Hubbs: https://towardsdatascience.com/social-distancing-to-slow-the-coronavirus-768292f04296

- S - Susceptible, E - Exposed, I - Infected, R - Recovered
- inputs: init_vals: S_0 = 1-1/n, E_0 = 1/N, I_0, R_0

- No death rate incoporated

In [ ]:
def base_seir_model(init_vals, params, t):
    S_0, E_0, I_0, R_0 = init_vals
    S, E, I, R = [S_0], [E_0], [I_0], [R_0]
    alpha, beta, gamma = params
    dt = t[1] - t[0]
    for _ in t[1:]:
        next_S = S[-1] - (beta*S[-1]*I[-1])*dt
        next_E = E[-1] + (beta*S[-1]*I[-1] - alpha*E[-1])*dt
        next_I = I[-1] + (alpha*E[-1] - gamma*I[-1])*dt
        next_R = R[-1] + (gamma*I[-1])*dt
        S.append(next_S)
        E.append(next_E)
        I.append(next_I)
        R.append(next_R)
    return np.stack([S, E, I, R]).T

- ρ, to capture our social distancing effect. This is going to be a constant term between 0–1, where 0 indicates everyone is locked down and quarantined while 1 is equivalent to our base case above.

In [ ]:
def seir_model_with_soc_dist(init_vals, params, t):
    S_0, E_0, I_0, R_0 = init_vals
    S, E, I, R = [S_0], [E_0], [I_0], [R_0]
    alpha, beta, gamma, rho = params
    dt = t[1] - t[0]
    for _ in t[1:]:
        next_S = S[-1] - (rho*beta*S[-1]*I[-1])*dt
        next_E = E[-1] + (rho*beta*S[-1]*I[-1] - alpha*E[-1])*dt
        next_I = I[-1] + (alpha*E[-1] - gamma*I[-1])*dt
        next_R = R[-1] + (gamma*I[-1])*dt
        S.append(next_S)
        E.append(next_E)
        I.append(next_I)
        R.append(next_R)
    return np.stack([S, E, I, R]).T

- So if we assume we have 10k people in our population, and we begin with one exposed person and the remaining 9,999 susceptible, we have:

In [ ]:
# Define parameters
t_max = 365
dt = 1
t = np.linspace(0, t_max, int(t_max/dt) + 1)
N = ny_pop
n = ny_confirmed
init_vals = 1 - n/N, n/N, 0, 0
alpha = 0.2
beta = 1.75
gamma = 0.5
params = alpha, beta, gamma
# Run simulation
results = base_seir_model(init_vals, params, t)
rho = 0.5
params = alpha, beta, gamma, rho
results2= seir_model_with_soc_dist(init_vals, params, t)

In [ ]:

plt.plot(pd.DataFrame(results)[0], label = "S")
plt.plot(pd.DataFrame(results)[1], label = "E")
plt.plot(pd.DataFrame(results)[2], label = "I")
plt.plot(pd.DataFrame(results)[3], label = "R")

plt.legend()

In [ ]:
def get_infections_number(t_max, dt, N, n,alpha,beta,gamma,rho):
    t = np.linspace(0, t_max, int(t_max/dt) + 1)
    init_vals = 1 - n/N, n/N, 0, 0
    params = alpha, beta, gamma, rho
    results = seir_model_with_soc_dist(init_vals, params, t)
    return pd.DataFrame(results)[2]*N

In [ ]:
r = get_infections_number(t_max=t_max,dt=dt,N=N, n=n, alpha=alpha, beta=beta, gamma=gamma,rho=rho)

In [ ]:
r_no_dist = get_infections_number(t_max=t_max,dt=dt,N=N, n=n, alpha=alpha, beta=beta, gamma=gamma,rho=1)

In [ ]:
plt.plot(r)
plt.plot(r_no_dist)

## Infections 
- What would the recover period for a infected people - 2 weeks 
- We need to be careful on identifying how many of infected people are in labor force - 60%


#### Infected number after 14 days (no quarantine)

In [ ]:
days = 14
labor_pct = 0.6

In [ ]:
ttl_infected = sum(r[:days])
post_comfirmed_cases = ny_confirmed + ttl_infected

In [ ]:
infection_rate = post_comfirmed_cases/ny_pop

In [ ]:
infection_rate

In [ ]:
labor_infection_rate = infection_rate*labor_pct

**Notes:**
-  we assumed 60% of infected case are in labor force
-  How about recovery? ppl who got sick 14 days ago should be recoverd from it.

- `labor_infection_rate` : how many of infected cases are in labor force.
    - We want to use this rate to identify how this would affect the manhours. This is a overall estimation. To be more granular, we could use adjust `labor_infection_rate` based on each industry 

In [ ]:
employment_ny.shape[0]

In [ ]:
employment_ny.industry

#### get `labor_infection_rate` for each industry
**Notes:**

1.  output = dollar_value_per_hour * num_hours 
    - Labor productivity :
        - data is not avaliable on states-industry level : only _Private Nonfarm_ https://www.bls.gov/lpc/lpc-by-state-and-region.xlsx
        - num_hours calculation: 
            - Hours worked
            - BLS creates estimates of state-level hours worked for all people as shown in equation (1):
            - Hours worked = [N * AWH * HWHP * 52] + SEUFWhours,                                                              (1)

    where N is wage and salary employment, AWH is all-employee average weekly hours paid, HWHP is the hours-worked-to-hours-paid ratio, and SEUFWhours is unincorporated self-employed worker (also referred to as proprietors) and unpaid family worker hours. For wage and salary workers, employment is based primarily on the CES state-level employment series, with supplemental employment counts for the nonfarm portion of the agricultural sector coming from the Quarterly Census of Employment and Wages (QCEW).18 In some instances in which QCEW data are suppressed, employment is imputed using a variety of methods, such as linear interpolation, ratios of the missing industry to the total in different years, or ratios based on establishment counts. In total over the period, approximately 25 percent of the state-year observations contain imputed data. However, the imputed estimates average only 0.005 percent of total wage and salary employment.
    
    
    
2. Simple approach:
    - Looking for labor reduction on state-industry level.
    - then apply that labor reduction to GDP reduction

In [ ]:
labor_infection_rate_v = np.dot(labor_infection_rate,np.repeat(1,employment_ny.shape[0]))

In [ ]:
employment_ny['2019']*labor_infection_rate_v